In [7]:
import sys
sys.path.append('..')

import torch
from torch.nn import BCELoss
from torch.optim import Adam
from torchvision.models import resnet18

from lib.models import SimpleClassifier, ResNet
from lib.trainer import Trainer
from lib.utils import train_parse_args, get_data_loaders
from lib.constants import HYPERPARAMETERS

In [8]:
params = HYPERPARAMETERS
params['batch_size'] = 8
params['verbose'] = True
params['version'] = 0.2

if not params['disable_cuda'] and torch.cuda.is_available():
    params['device'] = torch.device('cuda:0')
else:
    params['device'] = torch.device('cpu')
    
print(params)

{'batch_size': 8, 'checkpoints_dir': '/home/sergevkim/git/FacesClassification/checkpoints', 'checkpoint_filename': '', 'disable_cuda': False, 'imgs_dir': '/home/sergevkim/git/FacesClassification/data/CelebaHQ', 'label': 'Male', 'labels_filename': '/home/sergevkim/git/FacesClassification/data/list_attr_celeba.txt', 'logs_dir': '/home/sergevkim/git/FacesClassification/logs', 'n_epochs': 10, 'n_imgs': 30000, 'verbose': True, 'version': 0.2, 'device': device(type='cuda', index=0)}


In [ ]:
loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'])

model = ResNet()
model.cuda()

optimizer = Adam(model.parameters(), lr=3e-4)
criterion = BCELoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

0 0.75
100 0.625
200 0.375
300 0.75
! 0.5746666666666667
EPOCH 1
1 0 0.6832250296808893
1 100 -31.17459607014838
1 200 -45.283843058212504
1 300 25.42614936453704
1 400 -1.180168160916498
1 500 12.528511071231407
1 600 29.045723000147596
1 700 17.55178737255345
1 800 -16.22222279126888
1 900 -0.3794932644671434
1 1000 -18.52180241464996
1 1100 20.52749872980266
1 1200 5.311333655677739
1 1300 42.01354883478426
1 1400 -4.851733692269391
1 1500 1.7327079926326174
1 1600 -31.507779105240186
1 1700 -45.58531573257632
1 1800 47.08613968231177
1 1900 33.941740052283905
1 2000 -17.1968884499406
1 2100 -32.21384096753629
1 2200 -13.649475102677568
1 2300 34.4882516907499
1 2400 -29.20237589482871
1 2500 0.019412050962507976
1 2600 -15.884332190361357
1 2700 -24.80496930276932
1 2800 14.96028659446225
1 2900 -32.797074323228095
1 3000 31.472423995560447
1 3100 0.2307672336668709
1 3200 32.51789953857331
1 3300 -48.37406539862506
0 0.75
100 0.625
200 0.375
300 0.75
! 0.575
Epoch time: -1266.5922

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
a = np.array(Image.open("data/0.png"))
print(type(a))
plt.imshow(a)